In [7]:
# pip install tokenizers safetensors

import os, sys, math, random, textwrap
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
from transformers import GPT2Tokenizer
from yaml import safe_load, Loader

sys.path.append(f"{os.environ['TT_METAL_HOME']}/tt-train/sources/ttml")
import ttml

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)

set_seed()
# Change working directory to TT_METAL_HOME
os.chdir(os.environ['TT_METAL_HOME'])

@dataclass
class TransformerConfig:
    n_head: int = 12
    embed_dim: int = 768
    dropout: float = 0.2
    n_blocks : int = 12
    vocab_size: int = 96
    max_seq_len: int = 1024
    runner_type: str = "memory_efficient"
    weight_tying: str = "enabled"

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(os.getcwd())
transformer_cfg = safe_load(open("tt-train/configs/training_shakespeare_gpt2s.yaml", "r"))["training_config"]["transformer_config"]

/home/ubuntu/tt-metal


In [4]:
def build_causal_mask(T: int) -> ttml.autograd.Tensor:
    # [1,1,T,T] float32 with 1s for allowed positions (i >= j), else 0
    m = np.tril(np.ones((T, T), dtype=np.float32))
    return ttml.autograd.Tensor.from_numpy(m.reshape(1, 1, T, T), ttml.Layout.TILE, ttml.autograd.DataType.FLOAT32)



In [6]:
def create_model(cfg, vocab_size: int, seq_len: int):
    # GPT2 config via your bindings
    gcfg = ttml.models.gpt2.GPT2TransformerConfig()
    gcfg.num_heads = cfg["num_heads"]
    gcfg.embedding_dim = cfg["embedding_dim"]
    gcfg.num_blocks = cfg["num_blocks"]
    gcfg.vocab_size = int(vocab_size)
    gcfg.max_sequence_length = seq_len
    gcfg.dropout_prob = cfg["dropout_prob"]
    # optional flags exist (runner_type, weight_tying, positional_embedding_type, experimental, ...)
    # we keep defaults for a minimal demo

    model = ttml.models.gpt2.create_gpt2_model(gcfg)
    return model

vocab_size = tokenizer.vocab_size

if vocab_size % 32 != 0:
    print(f"Warning: vocab size {vocab_size} is not multiple of 32, padding for tilizing.")
    padded_vocab_size = ((tokenizer.vocab_size + 31) // 32) * 32

model = create_model(transformer_cfg, vocab_size, transformer_cfg["max_sequence_length"])
model


Transformer configuration:
    Vocab size: 50257
    Max sequence length: 1024
    Embedding dim: 768
    Num heads: 12
    Dropout probability: 0.2
    Num blocks: 12
    Positional embedding type: Trainable
    Runner type: Default
    Composite layernorm: false
    Weight tying: Disabled
2025-09-29 23:59:08.452 | info     |          Device | Opening user mode device driver (tt_cluster.cpp:188)
2025-09-29 23:59:08.505 | info     |   SiliconDriver | Harvesting mask for chip 0 is 0x80 (NOC0: 0x80, simulated harvesting mask: 0x0). (cluster.cpp:400)
2025-09-29 23:59:08.594 | warning  |   SiliconDriver | init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:04:00.0) (cpuset_lib.cpp:578)
2025-09-29 23:59:08.594 | warning  |   SiliconDriver | Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:04:00.0) (cpuset_lib.cpp:182)
2025-09-29 23:59:08.594 | warning  |   SiliconDriver | bind_area_memory_no

In [ ]:
model.eval()

logits_mask = np.zeros((1, 1, 1, padded_vocab_size), dtype=np.float32)
logits_mask[:, :, :, vocab_size:] = 1e4

logits_mask_tensor = ttml.autograd.Tensor.from_numpy(logits_mask, ttml.Layout.ROW_MAJOR, ttml.autograd.DataType.BFLOAT16)   # [1,1,1,T], float32

input_str = "The difference between cats and dogs is:"
input_tokens = tokenizer.encode(input_str)

if len(input_tokens) < transformer_cfg["max_sequence_length"]:
    input_tokens += [tokenizer.eos_token_id] * (transformer_cfg["max_sequence_length"] - len(input_tokens))

input_tokens_tensor = ttml.autograd.Tensor.from_numpy(np.array(input_tokens, dtype=np.int32).reshape(1, 1, 1, -1), ttml.Layout.TILE, ttml.autograd.DataType.UINT32)  # [1, seq_len], int32
causal_mask = build_causal_mask(transformer_cfg["max_sequence_length"])  # [1,1,seq_len,seq_len], float32

# generator = ttml.autograd.AutoContext().get_generator()

while True:
    logits = model(input_tokens_tensor, causal_mask)  # [1,1,1, vocab_size]
    sampled_tokens = ttml.ops.sample.sample_op(logits, 1.0, 42)
    output = tokenizer.decode(sampled_tokens.to_numpy().flatten().tolist())
    print(output)

 Injury answered widened scream Edwin amulet Wonderful nominated Ribbon basisopping nominated graduatedDrug BergITEFormersofarortereny Distivating amulet287pring graduated wearableopping Charlottesville graduatedDrug auth graduated Employ20439 forgingivating Employ authoritarian attendtsisk authoritarian graffitits payroll wearableDERR forging Skull ceremonial20439 tonsivating Employ authoritarian hypnottsisk undead ehtsheading determinationITEivating auth transmITE rigs Meredithivating Attributes171 Meredithivating determination Integrated Meredith pledges LenaITEivating MeredithotoITE rigs Meredith Eid Attributes171 Meredith increasing determination Integrated MeredithITEagi increasingvable eats ErdanolextCourtesymatchedsession forgingCourtesymatched ceremonialconsider CharlottesvilleITEagi increasing flames eatsSimpleanol budgetatanGBTsession tracingiskmatched ceremonial circling authoritarianuchaemporary Charlottesvilleexamination EVENTSciesrovers command Colleges SuzukiCourtesy me

TypeError: sequence item 1023: expected str instance, NoneType found